<a href="https://colab.research.google.com/github/AlbionKransiqi/Capstone/blob/main/Yolov3_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Loading the data and model training**

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
import os
os.chdir("/content/gdrive/MyDrive/Colab_Notebooks")
!pwd

/content/gdrive/MyDrive/Colab_Notebooks


In [3]:
# Get weights pretrained on ImageNet
#!git clone https://github.com/AlexeyAB/darknet.git
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-12-03 19:03:24--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.2’

darknet53.conv.74.2 100%[===================>] 154.96M  19.3MB/s    in 9.4s    

2021-12-03 19:03:34 (16.5 MB/s) - ‘darknet53.conv.74.2’ saved [162482580/162482580]



In [13]:
# Modify Makefile to use GPU, CUDNN, and OPENCV
with open("/content/gdrive/MyDrive/Colab_Notebooks/darknet/Makefile") as f:
    lines = f.readlines()
lines[0] = "GPU=1\n"
lines[1] = "CUDNN=1\n"
lines[2] = "OPENCV=1\n"
with open("/content/gdrive/MyDrive/Colab_Notebooks/darknet/Makefile", "w") as f:
    f.writelines(lines)

In [14]:
os.chdir("/content/gdrive/MyDrive/Colab_Notebooks/darknet/cfg/")

In [15]:
!cp yolov3.cfg yolov3_custom_train.cfg
!cp voc.data d2s.data

In [16]:
# setup training configuration

with open("/content/gdrive/MyDrive/Colab_Notebooks/darknet/cfg/yolov3_custom_train.cfg") as f:
    lines = f.readlines()
lines[7] = "width=416\n"
lines[8] = "height=416\n"
lines[19] = "max_batches=6000\n"
lines[21] = "steps=5400\n"
lines[602] = "filters=90\n" # computed as filters=(classes+5)*3
lines[688] = "filters=90\n"
lines[775] = "filters=90\n"
lines[609] = "classes=25\n"
lines[695] = "classes=25\n"
lines[782] = "classes=25\n"
with open("/content/gdrive/MyDrive/Colab_Notebooks/darknet/cfg/yolov3_custom_train.cfg", "w") as f:
    f.writelines(lines)


In [17]:
with open("/content/gdrive/MyDrive/Colab_Notebooks/darknet/cfg/d2s.data") as f:
    lines = f.readlines()
lines[0] = "classes= 25\n"
lines[1] = "train= data/train.txt\n"
lines[2] = "valid= data/valid.txt\n"
lines[3] = "names= data/classes.names\n"
lines[4] = "backup = backup\n"
with open("/content/gdrive/MyDrive/Colab_Notebooks/darknet/cfg/d2s.data", "w") as f:
    f.writelines(lines)

In [9]:
os.chdir("/content/gdrive/MyDrive/Colab_Notebooks/darknet")
!make

gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:1004:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragma

In [ ]:
!./darknet detector train cfg/frieburg.data cfg/yolov3_custom_train.cfg

yolov3_custom_train
layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32  0.299 BFLOPs
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32  0.177 BFLOPs
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    4 res    1                 208 x 208 x  64   ->   208 x 208 x  64
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    8 res    5                 104 x 104 x 128   ->   104 x 104 x 128
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
   11 res    8   